# Notebook based code
You can write code in python in the notebook, ship things from the notebook and get respones, outputs, logs.

## Flyte v1 notebook challenges
1. You cannot run "nested" `dynamic` etc in a notebook
2. Seeing logs, retrieving information and identifying information is hard

## Things to note!
1. Simple init
2. asyncio is native in a notebook
3. nested calls square calls hello
4. Observe phases
5. Observe logs

In [ ]:
import flyte

In [ ]:
flyte.init_from_config()

In [ ]:
env = flyte.TaskEnvironment(
    name="hello_world",
    image=flyte.Image.from_debian_base(),
)

In [ ]:
import asyncio
from typing import List


@env.task
async def say_hello(data: str, lt: List[int]) -> str:
    print(f"Hello, world! - {flyte.ctx().action}")
    return f"Hello {data} {lt}"


@env.task
async def square(i: int = 3) -> int:
    print(flyte.ctx().action)
    return i * i


@env.task
async def say_hello_nested(data: str = "default string") -> str:
    print(f"Hello, nested! - {flyte.ctx().action}", flush=True)
    coros = []
    for i in range(3):
        print(f"Enqueuing, Square({i})", flush=True)
        coros.append(square(i=i))

    vals = await asyncio.gather(*coros)
    print(f"Completed with vals = {vals}", flush=True)
    return await say_hello(data=data, lt=vals)

In [ ]:
run = await flyte.run.aio(say_hello_nested, data="jupyter says hello 2!")

In [ ]:
run.url

In [ ]:
await run.show_logs()

In [ ]:
run.name